In [56]:
import pandas as pd
import numpy as np 


In [ ]:
df = pd.read_csv('../data/raw/no_stock.csv')

In [ ]:
df.head()

,country_code,order_no,order_date,order_type,service_type,sales_channel,prime_line_no,line_status_code,line_status_description,item_id,...,time_to_min_notification_date_days,time_to_promised_appt_start_date_mins,time_to_promised_appt_start_date_hours,time_to_promised_appt_start_date_days,time_to_dispatch_date_mins,time_to_dispatch_date_hours,time_to_dispatch_date_days,deviation,ship_node_deviation_rate_1month,ship_node_stock_related_deviation_rate_1month
0,DE,1556071560,2025-10-26 11:20:21+00:00,HOME_DELIVERY,HOME_DELIVERY,INTERNET,3,1500.02,Allocated,ART70521030,...,1.638727,3979.766667,66.329444,2.763727,2839.766667,47.329444,1.972060,0,5.557888,1.077636
1,IT,1553618414,2025-10-15 14:19:46+00:00,HOME_DELIVERY,HOME_DELIVERY,INTERNET,2,1100.00,Created,ART90095428,...,NaN,11320.233333,188.670556,7.861273,9460.233333,157.670556,6.569606,0,20.058782,1.157086
2,FR,1553542398,2025-10-15 07:54:34+00:00,HOME_DELIVERY,HOME_DELIVERY,INTERNET,7,3200.02,Sent for fulfillment,ART70521030,...,1.828183,7072.583333,117.876389,4.911516,3172.583333,52.876389,2.203183,0,18.076389,0.972423
3,NL,1554605939,2025-10-16 16:02:08+00:00,HOME_DELIVERY,HOME_DELIVERY,FAX,40,3200.02,Sent for fulfillment,ART20597918,...,5.162384,9053.833333,150.897222,6.287384,8393.833333,139.897222,5.829051,0,4.932107,0.971959
4,BE,1551018238,2025-10-04 15:04:28+00:00,HOME_DELIVERY,HOME_DELIVERY,STORE,6,3200.02,Sent for fulfillment,ART50454036,...,4.036887,8753.116667,145.885278,6.078553,7193.116667,119.885278,4.995220,0,5.125719,3.999987


In [ ]:
#print every column name with number and type
for i, col in enumerate(df.columns):
    print(i, col, df[col].dtype)

0 country_code object
1 order_no int64
2 order_date object
3 order_type object
4 service_type object
5 sales_channel object
6 prime_line_no int64
7 line_status_code float64
8 line_status_description object
9 item_id object
10 ship_node object
11 ordered_qty float64
12 line_total_euros float64
13 transport_method_type object
14 item_group_code object
15 latest_dispatch_date object
16 work_order_no int64
17 cancellation_date object
18 delivery_date object
19 line_status_date object
20 no_of_DS int64
21 no_of_PS int64
22 payment_type object
23 payment_due_at object
24 min_notification_time float64
25 customer_type object
26 promised_update_DS int64
27 promised_update_PS int64
28 WO_creation_date object
29 WO_acknowledgement_date object
30 payment_fully_paid_date object
31 next_line_status_date object
32 min_notification_date object
33 forecast_stock_current_day float64
34 forecast_volume_current_day float64
35 forecast_stock_previous_day float64
36 forecast_volume_previous_day float64
37 

In [ ]:
#print value counts also for nans
print(df['event_type'].value_counts(dropna=False))

event_type
NaN              5057762
Auto-recovery       8367
Deviation           2145
Name: count, dtype: int64


In [ ]:
#print nans for every column with percentage in descending order
nan_counts = []
for col in df.columns:
    nans = df[col].isna().sum()
    if nans > 0:
        nan_counts.append((col, nans, nans/len(df)*100))

nan_counts.sort(key=lambda x: x[1], reverse=True)

for col, nans, perc in nan_counts:
    print(f"{col}: {nans} nans ({perc:.2f}%)")

customer_type: 8302 nans (0.16%)
prod_name_no: 2 nans (0.00%)
prod_name: 2 nans (0.00%)


In [ ]:
#drop columns with more than 90% nans
df = df.loc[:, df.isna().mean() < 0.9]

In [ ]:
# for "days_since_last_stockout" column, make all nans max value + 1
max_days = df['days_since_last_stockout'].max()
df['days_since_last_stockout'] = df['days_since_last_stockout'].fillna(max_days + 1)

max_days = df['estimated_days_till_next_stockout'].max()
df['estimated_days_till_next_stockout'] = df['estimated_days_till_next_stockout'].fillna(max_days + 1)

In [ ]:
#drop unuseable columns
to_drop = [
    'delivery_date',
    'modify_date_time',
]

df = df.drop(columns=to_drop)

In [ ]:
# TEMPORARY SOLUTION
# replace item_id_service_level nans with 5.0
df['item_id_service_level'] = df['item_id_service_level'].fillna(5.0)

In [ ]:
#TEMPORARY SOLUTION
# categories
# replace categorical nans with 'Unknown'
df['payment_type'] = df['payment_type'].fillna('Unknown')
df['customer_type'] = df['customer_type'].fillna('Unknown')

In [ ]:
#TEMPORARY SOLUTION
# numerical
# replace numerical nans with median
numerical_cols = [
    'forecast_volume_next_day', 
    'forecast_stock_next_day', 
    'forecast_volume_current_day', 
    'forecast_volume_previous_day', 
    'forecast_stock_current_day', 
    'forecast_stock_previous_day', 
    'days_since_last_replenishment', 
    'days_since_last_stockout',
    'current_daily_stock_change_rate',
    'average_daily_stock_change_rate',
    'average_daily_stock_change_rate_replenishment',
    'availability_qty',
    'average_stock_4_weeks',
    'stock_variance_4_weeks',
    'stockout_count_28_days',
    'replenishment_count_28_days',
    ]

for col in numerical_cols:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)
    

In [ ]:
date_cols = [
    'order_date',
    'payment_due_at',
    'payment_fully_paid_date',
    'min_notification_date',
    'WO_creation_date',
    'WO_acknowledgement_date',
    'latest_dispatch_date',
    # 'promised_appt_start_date_DS',
    # 'promised_appt_end_date_DS',
    'min_notification_time',

]

#drop for now
df = df.drop(columns=date_cols)

In [ ]:
time_cols = [
    'time_to_min_notification_date_mins',
    'time_to_min_notification_date_hours',
    'time_to_min_notification_date_days',
    'time_to_wo_creation_date_mins',
    'time_to_wo_creation_date_hours',
    'time_to_wo_creation_date_days',
    'time_to_WO_acknowledgement_date_mins',
    'time_to_WO_acknowledgement_date_hours',
    'time_to_WO_acknowledgement_date_days',
    'time_to_promised_appt_start_date_mins',
    'time_to_promised_appt_start_date_hours',
    'time_to_promised_appt_start_date_days',
]

#fill nans with median
for col in time_cols:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)

In [ ]:
#drop all remaining rows with nans
df = df.dropna()

In [ ]:
#calculate the ratio of nans with deviation=1 for each column that has nans in descending order
nan_deviation_counts = []
for col in df.columns:
    if df[col].isna().sum() > 0:
        nan_deviation_1 = df[df['deviation'] == 1][col].isna().sum()
        total_deviation_1 = len(df[df['deviation'] == 1])
        ratio_1 = nan_deviation_1 / total_deviation_1 if total_deviation_1 > 0 else 0
        # print(f"{col}: Deviation=1 NaN Ratio: {ratio_1:.4f}")
        nan_deviation_counts.append((col, ratio_1))

nan_deviation_counts.sort(key=lambda x: x[1], reverse=True)
for col, ratio_1 in nan_deviation_counts:
    print(f"{col}: Deviation=1 NaN Ratio: {ratio_1:.4f}")

customer_type: Deviation=1 NaN Ratio: 0.0004
prod_name_no: Deviation=1 NaN Ratio: 0.0000
prod_name: Deviation=1 NaN Ratio: 0.0000


In [ ]:
df['customer_type'].value_counts(dropna=False)

customer_type
PRIVATE     4759956
BUSINESS     300016
NaN            8302
Name: count, dtype: int64

In [ ]:
#count distinct pairs of (order_no, prime_line_no)
distinct_pairs = df[['order_no', 'prime_line_no']].drop_duplicates()
print(f"Distinct (order_no, prime_line_no) pairs: {len(distinct_pairs)}")

Distinct (order_no, prime_line_no) pairs: 1774094
